In [ ]:
import geopandas as gpd
import momepy as mm
import osmnx as ox
import numpy as np
from shapely.geometry import box
import utils

In [ ]:
buildings = gpd.read_file()

In [ ]:
boundary = gpd.GeoSeries([box(*buildings.total_bounds).buffer(200)], crs=buildings.crs).to_crs(4326).iloc[0]

In [ ]:
G = ox.graph_from_polygon(boundary, network_type='all')
edges = ox.graph_to_gdfs(G_proj, nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True).to_crs(buildings.crs)

In [ ]:
edges = mm.remove_false_nodes(edges.explode().reset_index(drop=True))

In [ ]:
edges.to_parquet('../../nairobi/edges.pq')

## Preprocess buildings

In [ ]:
buildings.geometry = buildings.buffer(0)
buildings = buildings.reset_index(drop=True).explode().reset_index(drop=True)

In [ ]:
buildings.geom_type.value_counts()

In [ ]:
buildings = buildings[buildings.area > 10]

In [ ]:
buildings = gpd.GeoDataFrame(geometry=utils.fill_insides(buildings))
buildings["uID"] = range(len(buildings))
buildings.crs

In [ ]:
buildings.to_parquet('../../nairobi/buildings.pq')

## Generate tessellation

In [ ]:
check = mm.CheckTessellationInput(buildings)

In [ ]:
# add removal

In [ ]:
limit = mm.buffered_limit(buildings, 100)
tess = mm.Tessellation(buildings, "uID", limit, segment=2).tessellation

In [ ]:
tess.to_parquet('../../nairobi/tessellation.pq')

## Generate blocks

In [ ]:
extended = mm.extend_lines(edges, 20)

In [ ]:
blocks, b_id, t_id = utils.new_blocks(tess, extended, buildings, id_name='bID', unique_id='uID')
buildings['bID'] = b_id
tessellation['bID'] = t_id

In [ ]:
blocks.to_parquet('../../nairobi/blocks.pq')

## Link streets

In [ ]:
edges["nID"] = range(len(edges))
buildings['nID'] = mm.get_network_id(buildings, edges, 'nID', min_size=500)  
tess = tess.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [ ]:
tess.to_parquet('../../nairobi/tessellation.pq')
buildings.to_parquet('../../nairobi/buildings.pq')
edges.to_parquet('../../nairobi/edges.pq')